<div class="alert alert-block alert-info">
<h1 style="text-align:center;font-family:avenir;">GLM second level</h1>
Lou Planchamp & Victor Férat
</div>


In [2]:
%matplotlib inline

In [3]:
from nilearn import plotting
from nilearn.plotting import plot_stat_map
import pandas as pd
from nilearn.glm.first_level import FirstLevelModel
from numpy import array
import os
from nilearn.plotting import plot_design_matrix
import numpy as np
import matplotlib.pyplot as plt

from nilearn.glm.second_level import SecondLevelModel

Run this code to compute a second level analisis on the t-maps of all subjects and all sessions at once 

In [9]:
#ALL sub ALL ses

anat = '/home/carole/Bureau/Lou_dualreg/tpl-MNIPediatricAsym_cohort-3_res-1_T1w.nii'

if not os.path.isdir('sec_level_GLM'):
    os.mkdir('sec_level_GLM')
if not os.path.isdir('sec_level_GLM/zmaps'): 
    os.mkdir('sec_level_GLM/zmaps')

for task in ['Transfer', 'Calcul', 'CPT', 'Memory']:
    zfolder = f'sec_level_GLM/zmaps/{task}'
    if not os.path.isdir(zfolder): 
        os.mkdir(zfolder)
    print(task)
    
    #====================
   
    first_t_maps = dict()
    folder = f'first_level_GLM/tmaps/{task}'
    tmaps = []

    for file in os.listdir(folder):
        if file.endswith('.nii'):
                tmaps.append(os.path.join(folder,file))

    first_t_maps['tmaps'] = tmaps
    tmaps.sort()
    
    #=====================

    fig_folder = f'sec_level_GLM/zmaps/fig'
    if not os.path.isdir(fig_folder): 
        os.mkdir(fig_folder)
    all_t_maps_folder = f'sec_level_GLM/zmaps/fig/all_t_maps'
    if not os.path.isdir(all_t_maps_folder): 
        os.mkdir(all_t_maps_folder)
    fig_name = f'sec_level_GLM/zmaps/fig/all_t_maps/fig_all_t_maps_{task}'

    subjects = [os.path.basename(file).split('_')[0] for file in tmaps]
    fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(8,10))
    for cidx, tmap in enumerate(first_t_maps['tmaps']):
        plotting.plot_glass_brain(
            tmap,
            colorbar=True,
    #        threshold=3,
            title=subjects[cidx],
            annotate=False,
            axes=axes[int(cidx / 2), int(cidx % 2)],
            plot_abs=False,
            display_mode='z',
            vmax=11, #change for no comparision btw sub and tasks
            output_file=fig_name)

    #=======================
    
    second_level_input = first_t_maps['tmaps']
    design_matrix = pd.DataFrame([1] * len(second_level_input), columns=['intercept'])

    second_level_model = SecondLevelModel(smoothing_fwhm=8.0)
    second_level_model = second_level_model.fit(
        second_level_input,
        design_matrix=design_matrix)
    
    sec_z_map = second_level_model.compute_contrast(output_type='z_score')

    zpath = os.path.join(zfolder, f'sec_z_map_all-sub_all-ses_{task}.nii')
    sec_z_map.to_filename(zpath)
    
    sec_z_maps_folder = f'sec_level_GLM/zmaps/fig/sec_z_maps_all-sub_all-ses'
    if not os.path.isdir(sec_z_maps_folder): 
        os.mkdir(sec_z_maps_folder)
    zfig_name = f'sec_level_GLM/zmaps/fig/sec_z_maps_all-sub_all-ses/fig_all-sub_all-ses_sec_z_map_{task}'
    
    plotting.plot_stat_map(sec_z_map, display_mode='mosaic',
                           threshold=2, bg_img=anat, 
                           output_file=zfig_name,
                           annotate=False, black_bg=True, cut_coords=8,
                           title=f'z-maps {task} all-sub all-ses')
    

Transfer
Calcul
CPT
Memory


This code is the same, second level with all sessions at once but excluding sub 5 and 7 



In [12]:
# ALL ses without sub 5 and 7

anat = '/home/carole/Bureau/Lou_dualreg/tpl-MNIPediatricAsym_cohort-3_res-1_T1w.nii'

if not os.path.isdir('sec_level_GLM'):
    os.mkdir('sec_level_GLM')
if not os.path.isdir('sec_level_GLM/zmaps'): 
    os.mkdir('sec_level_GLM/zmaps')

for task in ['Transfer', 'Calcul', 'CPT', 'Memory']:
    zfolder = f'sec_level_GLM/zmaps/{task}'
    if not os.path.isdir(zfolder): 
        os.mkdir(zfolder)
    print(task)
    
    #====================
   
    first_t_maps = dict()
    folder = f'first_level_GLM/tmaps/{task}'
    tmaps = []

    for file in os.listdir(folder):
        if file.endswith('.nii'):
            if not file.startswith('sub-05') and not file.startswith('sub-07'):
                tmaps.append(os.path.join(folder,file))

    first_t_maps['tmaps'] = tmaps
    tmaps.sort()
    
    #=====================
        
    fig_folder = f'sec_level_GLM/zmaps/fig'
    if not os.path.isdir(fig_folder): 
        os.mkdir(fig_folder)
    all_t_maps_folder = f'sec_level_GLM/zmaps/fig/all_t_maps'
    if not os.path.isdir(all_t_maps_folder): 
        os.mkdir(all_t_maps_folder)
    fig_name = f'sec_level_GLM/zmaps/fig/all_t_maps/fig_all_t_maps_wo-sub_{task}'

    subjects = [os.path.basename(file).split('_')[0] for file in tmaps]
    # nrows = 3 because we have 3 sub
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(8,10))
    for cidx, tmap in enumerate(first_t_maps['tmaps']):
        plotting.plot_glass_brain(
            tmap,
            colorbar=True,
    #        threshold=3,
            title=subjects[cidx],
            annotate=False,
            axes=axes[int(cidx / 2), int(cidx % 2)],
            plot_abs=False,
            display_mode='z',
            vmax=10, #change for no comparision btw sub and tasks
            output_file=fig_name)

    #=======================
    
    second_level_input = first_t_maps['tmaps']
    design_matrix = pd.DataFrame([1] * len(second_level_input), columns=['intercept'])

    second_level_model = SecondLevelModel(smoothing_fwhm=8.0)
    second_level_model = second_level_model.fit(
        second_level_input,
        design_matrix=design_matrix)
    
    sec_z_map = second_level_model.compute_contrast(output_type='z_score')

    zpath = os.path.join(zfolder, f'sec_z_map_wo-sub_all-ses_{task}.nii')
    sec_z_map.to_filename(zpath)
    
    zfig_name = f'sec_level_GLM/zmaps/fig/fig_wo-sub_all-ses_sec_z_map_{task}'
    
    sec_z_maps_folder = f'sec_level_GLM/zmaps/fig/sec_z_maps_wo-sub_all-ses'
    if not os.path.isdir(sec_z_maps_folder): 
        os.mkdir(sec_z_maps_folder)
    zfig_name = f'sec_level_GLM/zmaps/fig/sec_z_maps_wo-sub_all-ses/fig_wo-sub_all-ses_sec_z_map_{task}'
    
    plotting.plot_stat_map(sec_z_map, display_mode='mosaic',
                           threshold=2, bg_img=anat, 
                           output_file=zfig_name,
                           annotate=False, black_bg=True, cut_coords=8,
                           title=f'z-maps {task} wo-sub all-ses')
    

Transfer
Calcul
CPT
Memory


Run this code to compute a second level analisis on the t-maps of all subjects for the pre and post sessions separatly 

(note that there is just the post session for the transfer run, so the results are the same than with the precedent code)

In [18]:
# SES pre-post ALL sub

if not os.path.isdir('sec_level_GLM'):
    os.mkdir('sec_level_GLM')
if not os.path.isdir('sec_level_GLM/zmaps'): 
    os.mkdir('sec_level_GLM/zmaps')

for task in ['Transfer', 'Calcul', 'CPT', 'Memory']:
    for session in ['02', '11']:
        if task == 'Transfer':
            session = '11'
            # Not ideal because run twice but it works, just takes a bit more time 
            # To be modify
            
        zfolder = f'sec_level_GLM/zmaps/{task}'
        if not os.path.isdir(zfolder): 
            os.mkdir(zfolder)
        print(task)
        print(session)

        #====================

        first_t_maps = dict()
        folder = f'first_level_GLM/tmaps/{task}'
        tmaps = []

        for file in os.listdir(folder):
            if file.endswith(f'{session}_{task}_tmap.nii'):
                tmaps.append(os.path.join(folder,file))

        first_t_maps['tmaps'] = tmaps
        tmaps.sort()

        #=====================

        fig_folder = f'sec_level_GLM/zmaps/fig'
        if not os.path.isdir(fig_folder): 
            os.mkdir(fig_folder)
        all_t_maps_folder = f'sec_level_GLM/zmaps/fig/all_t_maps'
        if not os.path.isdir(all_t_maps_folder): 
            os.mkdir(all_t_maps_folder)
        ses_folder = f'sec_level_GLM/zmaps/fig/all_t_maps/all-sub_ses-prepost'
        if not os.path.isdir(ses_folder): 
            os.mkdir(ses_folder)
        fig_name = f'sec_level_GLM/zmaps/fig/all_t_maps/all-sub_ses-prepost/fig_t_maps_all-sub_ses-{session}_{task}'

        subjects = [os.path.basename(file).split('_')[0] for file in tmaps]
        # nrows = 3 because we have 5 sub on just one session
        fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(8,10))
        for cidx, tmap in enumerate(first_t_maps['tmaps']):
            plotting.plot_glass_brain(
                tmap,
                colorbar=True,
        #        threshold=3,
                title=subjects[cidx],
                annotate=False,
                axes=axes[int(cidx / 2), int(cidx % 2)],
                plot_abs=False,
                display_mode='z',
        #        vmax=11,
                output_file=fig_name)
        fig.suptitle(f'fig_t_maps_ses-{session}_{task}')

        #=======================

        second_level_input = first_t_maps['tmaps']
        design_matrix = pd.DataFrame([1] * len(second_level_input), columns=['intercept'])

        
        second_level_model = SecondLevelModel(smoothing_fwhm=8.0)
        second_level_model = second_level_model.fit(
            second_level_input,
            design_matrix=design_matrix)

        sec_z_map = second_level_model.compute_contrast(output_type='z_score')

        zpath = os.path.join(zfolder, f'sec_z_map_all-sub_ses-{session}_{task}.nii')
        sec_z_map.to_filename(zpath)

        sec_z_maps_folder = f'sec_level_GLM/zmaps/fig/sec_z_maps_all-sub_ses-prepost'
        if not os.path.isdir(sec_z_maps_folder): 
            os.mkdir(sec_z_maps_folder)
        zfig_name = f'sec_level_GLM/zmaps/fig/sec_z_maps_all-sub_ses-prepost/fig_all-sub_ses-{session}_sec_z_map_{task}'

        plotting.plot_stat_map(sec_z_map, display_mode='mosaic',
                               threshold=2, bg_img=anat, 
                               output_file=zfig_name,
                               annotate=False, black_bg=True, cut_coords=8,
                               title=f'z-maps {task} all-sub ses-{session} ')


Transfer
11
Transfer
11
Calcul
02
Calcul
11
CPT
02
CPT
11
Memory
02
Memory
11


Run this code to compute a second level analisis on the t-maps for the pre and post sessions separatly without sub 5 and 7

(note that there is just the post session for the transfer run, so the results are the same than with the precedent code)

In [20]:
# SES pre-post WO sub 5 and 7

if not os.path.isdir('sec_level_GLM'):
    os.mkdir('sec_level_GLM')
if not os.path.isdir('sec_level_GLM/zmaps'): 
    os.mkdir('sec_level_GLM/zmaps')

for task in ['Transfer', 'Calcul', 'CPT', 'Memory']:
    for session in ['02', '11']:
        if task == 'Transfer':
            session = '11'
            # Not ideal because run twice but it works, just takes a bit more time 
            # To be modify
            
        zfolder = f'sec_level_GLM/zmaps/{task}'
        if not os.path.isdir(zfolder): 
            os.mkdir(zfolder)
        print(task)
        print(session)

        #====================

        first_t_maps = dict()
        folder = f'first_level_GLM/tmaps/{task}'
        tmaps = []

        for file in os.listdir(folder):
            if file.endswith(f'{session}_{task}_tmap.nii'):
                if not file.startswith('sub-05') and not file.startswith('sub-07'):
                    tmaps.append(os.path.join(folder,file))

        first_t_maps['tmaps'] = tmaps
        tmaps.sort()

        #=====================

        fig_folder = f'sec_level_GLM/zmaps/fig'
        if not os.path.isdir(fig_folder): 
            os.mkdir(fig_folder)
        all_t_maps_folder = f'sec_level_GLM/zmaps/fig/all_t_maps'
        if not os.path.isdir(all_t_maps_folder): 
            os.mkdir(all_t_maps_folder)
        ses_folder = f'sec_level_GLM/zmaps/fig/all_t_maps/wo-sub_ses-prepost'
        if not os.path.isdir(ses_folder): 
            os.mkdir(ses_folder)
        fig_name = f'sec_level_GLM/zmaps/fig/all_t_maps/all-sub_ses-prepost/fig_t_maps_wo-sub_ses-{session}_{task}'

        subjects = [os.path.basename(file).split('_')[0] for file in tmaps]
        # nrows = 2 because we have 3 sub on just one session
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8,10))
        for cidx, tmap in enumerate(first_t_maps['tmaps']):
            plotting.plot_glass_brain(
                tmap,
                colorbar=True,
        #        threshold=3,
                title=subjects[cidx],
                annotate=False,
                axes=axes[int(cidx / 2), int(cidx % 2)],
                plot_abs=False,
                display_mode='z',
        #        vmax=11,
                output_file=fig_name)
        fig.suptitle(f'fig_t_maps_ses-{session}_{task}')

        #=======================

        second_level_input = first_t_maps['tmaps']
        design_matrix = pd.DataFrame([1] * len(second_level_input), columns=['intercept'])

        
        second_level_model = SecondLevelModel(smoothing_fwhm=8.0)
        second_level_model = second_level_model.fit(
            second_level_input,
            design_matrix=design_matrix)

        sec_z_map = second_level_model.compute_contrast(output_type='z_score')

        zpath = os.path.join(zfolder, f'sec_z_map_wo-sub_ses-{session}_{task}.nii')
        sec_z_map.to_filename(zpath)

        sec_z_maps_folder = f'sec_level_GLM/zmaps/fig/sec_z_maps_wo-sub_ses-prepost'
        if not os.path.isdir(sec_z_maps_folder): 
            os.mkdir(sec_z_maps_folder)
        zfig_name = f'sec_level_GLM/zmaps/fig/sec_z_maps_wo-sub_ses-prepost/fig_wo-sub_ses-{session}_sec_z_map_{task}'

        plotting.plot_stat_map(sec_z_map, display_mode='mosaic',
                               threshold=2, bg_img=anat, 
                               output_file=zfig_name,
                               annotate=False, black_bg=True, cut_coords=8,
                               title=f'z-maps {task} wo-sub ses-{session}')


Transfer
11
Transfer
11
Calcul
02
Calcul
11
CPT
02
CPT
11
Memory
02
Memory
11
